In [26]:
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import sys
sys.path.append("../model/")
from dataset import *
from MLP_classifier import MultiClassClassifier
import torch
from tqdm import tqdm

from pathlib import Path

path_meta = "/data3/TEST/A/" #Rémi: change the directory
tmp_dir = "/data4/saland/tmp/" #All: change the directory, this is for augmented images

In [41]:
device = "cpu"
model = MultiClassClassifier(n_classes=2).to(device)
model.load_state_dict(torch.load("../model/checkpoints/binary_train_real_fake_2k_fine_tune_meta_test.pt"))
model.eval()
data = TaskA(load_from_disk=True,path="/data4/saland/data/taskA.pt")

In [42]:
path = Path(path_meta)
im_path = sorted(list(path.rglob('*.jpg'))+list(path.rglob('*.jpeg'))+list(path.rglob('*.png')))
im_names = [p.name for p in im_path]
im_dir = [str(p.parents[0]) for p in im_path]

df = pd.DataFrame(columns=['dir_name','image_name','soft_max'])
df['image_name'] = im_names
df['dir_name'] = im_dir

df.head()


,dir_name,image_name,soft_max
0,/data3/TEST/A,A_005fbfn6.png,NaN
1,/data3/TEST/A,A_0060ug9j.png,NaN
2,/data3/TEST/A,A_00el4hwr.png,NaN
3,/data3/TEST/A,A_00h6ucsm.png,NaN
4,/data3/TEST/A,A_00hzao5p.png,NaN


In [43]:
sm_list = []
for index, row in df.iterrows():
    # im_name= str(row['dir_name'])+'/'+(row['image_name'])
    with torch.no_grad():
        softmax = torch.softmax(model(data.features[row['image_name']].to(device)),dim=1)
    sm_list.append(softmax.cpu().numpy().flatten())

df['soft_max']= sm_list
df.head()
    

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [37]:
#df.drop('dir_name')
with np.printoptions(linewidth=10000):
    df.to_csv("methods_logits.csv")
df.head()


,dir_name,image_name,soft_max
0,/data3/TEST/A,A_005fbfn6.png,"[0.9831618, 0.016838139]"
1,/data3/TEST/A,A_0060ug9j.png,"[1.0, 1.03741536e-10]"
2,/data3/TEST/A,A_00el4hwr.png,"[1.3547017e-07, 0.9999999]"
3,/data3/TEST/A,A_00h6ucsm.png,"[0.9973826, 0.0026174497]"
4,/data3/TEST/A,A_00hzao5p.png,"[2.1300623e-07, 0.99999976]"


In [38]:
df.to_csv("../../misc/clip_softmax_A.csv")